## Deterministic set-up

In [1]:
import numpy as np

In [2]:
# Encode States as numbers by declaring a dictionary

state = {
    0 : "Susceptible",
    1 : "Exposed",
    2 : "Infectious_0",
    3 : "Infectious_1",
    4 : "Infectious_2",
    5 : "Infectious_3",
    6 : "Infectious_4",
    7 : "Recovered",
    8 : "Dead",
}

# Create a numpy matrix "A" containing all transition probabilites
# Matrix referencing == "A_ij", where the entry is the transition 
# probability from "State i" to "State j"
# Note matrix must be square (i length is j length)

trprob = np.array([
    [0.69, 0.31, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0.25, 0.75, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0.19, 0, 0, 0.81, 0],
    [0, 0, 0, 0, 0, 0.31, 0, 0.69, 0],
    [0, 0, 0, 0, 0, 0, 0.12, 0.53, 0.35],
    [0, 0, 0, 0, 0, 0, 0, 0.65, 0.35],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
])

### Deterministic, no-intervention model

In [3]:
# Random Walk on Markov Chain
# What a single run looks like

n = 20
start_state = 0
curr_state = start_state
print(state[curr_state], "--->", end=" ")

while n-1:
    curr_state = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], p=trprob[curr_state])
    print(state[curr_state], "--->", end=" ")
    n-=1
print("stop")

Susceptible ---> Susceptible ---> Susceptible ---> Susceptible ---> Exposed ---> Infectious_1 ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> Recovered ---> stop


GPT Code

In [4]:
'''
Monte Carlo Method to repeat simulations
Simply "tallying"
the amount of times individuals appeared in each state
'''

# Number of simulations
simulations = 10**4
n = 20  # Number of steps per simulation

# Initialize count array
state_counts = [0, 0, 0, 0, 0, 0, 0, 0, 0]

# Run simulations
for _ in range(simulations):
    curr_state = 0  # Start from Susceptible
    state_counts[curr_state] += 1  # Count the initial state
    
    steps = n - 1  # Remaining steps
    while steps > 0:
        curr_state = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], p=trprob[curr_state])
        state_counts[curr_state] += 1
        steps -= 1

# Print results
for idx, count in enumerate(state_counts):
    print(f"{state[idx]}: {count}")

Susceptible: 32121
Exposed: 9993
Infectious_0: 2490
Infectious_1: 7499
Infectious_2: 1389
Infectious_3: 417
Infectious_4: 53
Recovered: 143741
Dead: 2297


In [5]:
scts = state_counts

state_cost_totals = [
    scts[0]*0, 
    scts[1]*0, 
    scts[2]*0, 
    scts[3]*123.81, 
    scts[4]*12313.27, 
    scts[5]*40168.43, 
    scts[6]*40168.43, 
    scts[7]*0, 
    scts[8]*0, 
]

state_cost_totals

[0, 0, 0, 928451.1900000001, 17103132.03, 16750235.31, 2128926.79, 0, 0]

In [6]:
total_cost = sum(state_cost_totals)
total_cost

36910745.32

In [7]:
'''
Doing the same but for QALYs
'''

state_util_totals = [
    scts[0]*0.880, 
    scts[1]*0.880, 
    scts[2]*0.833, 
    scts[3]*0.614, 
    scts[4]*0.500, 
    scts[5]*0.250, 
    scts[6]*0.050, 
    scts[7]*0.880, 
    scts[8]*0, 
]

state_util_totals

[28266.48,
 8793.84,
 2074.17,
 4604.3859999999995,
 694.5,
 104.25,
 2.6500000000000004,
 126492.08,
 0]

In [8]:
total_utility = sum(state_util_totals)
total_utility

171032.356

### Deterministic vaccine intervention model

In [9]:
'''
Monte Carlo Method for vaccines
'''

# only 4000 people because 60% of total pop is vaccinated
vacc_simulations = 4**3
k = 20

# Initialize with 6000 people already recovered
v_state_counts = [0, 0, 0, 0, 0, 0, 0, 120000, 0]

# Run simulations
for _ in range(vacc_simulations):
    v_curr_state = 0  # Start from Susceptible
    v_state_counts[v_curr_state] += 1  # Count the initial state
    
    v_steps = k - 1  # Remaining steps
    while v_steps > 0:
        v_curr_state = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], p=trprob[v_curr_state])
        v_state_counts[v_curr_state] += 1
        v_steps -= 1

# Print results
for idx, count in enumerate(v_state_counts):
    print(f"{state[idx]}: {count}")

Susceptible: 188
Exposed: 64
Infectious_0: 13
Infectious_1: 51
Infectious_2: 8
Infectious_3: 3
Infectious_4: 0
Recovered: 120939
Dead: 14


In [10]:
vcts = v_state_counts

vacc_cost_totals = [
    vcts[0]*0, 
    vcts[1]*0, 
    vcts[2]*0, 
    vcts[3]*123.81, 
    vcts[4]*12313.27, 
    vcts[5]*40168.43, 
    vcts[6]*40168.43, 
    vcts[7]*0, 
    vcts[8]*0, 
]

vacc_cost_totals

[0, 0, 0, 6314.31, 98506.16, 120505.29000000001, 0.0, 0, 0]

In [11]:
vacc_cost = sum(vacc_cost_totals) + 6000*65.87
vacc_cost

620545.76

In [12]:
'''
Doing the same but for QALYs
'''

vacc_util_totals = [
    vcts[0]*0.880, 
    vcts[1]*0.880, 
    vcts[2]*0.833, 
    vcts[3]*0.614, 
    vcts[4]*0.500, 
    vcts[5]*0.250, 
    vcts[6]*0.050, 
    vcts[7]*0.880, 
    vcts[8]*0, 
]

vacc_util_totals

[165.44, 56.32, 10.828999999999999, 31.314, 4.0, 0.75, 0.0, 106426.32, 0]

In [13]:
vacc_utility = sum(vacc_util_totals) + 120000*0.9
vacc_utility

214694.973

### Deterministic ICER

In [14]:
det_num = vacc_cost - total_cost
det_deno = vacc_utility - total_utility

det_ICER = det_num/det_deno
det_ICER

-831.1503536308876

In [15]:
print("cost delta =", det_num)
print("QALY delta =", det_deno)

cost delta = -36290199.56
QALY delta = 43662.617


Really suggests that it lies in the fourth quadrant of delta cost vs delta QALY.
That means it's by default cost effective.

## Probabilistic set-up

In [16]:
# Encode States as numbers by declaring a dictionary

state = {
    0 : "Susceptible",
    1 : "Exposed",
    2 : "Infectious_0",
    3 : "Infectious_1",
    4 : "Infectious_2",
    5 : "Infectious_3",
    6 : "Infectious_4",
    7 : "Recovered",
    8 : "Dead",
}

# Create a numpy matrix "A" containing all transition probabilites
# Matrix referencing == "A_ij", where the entry is the transition 
# probability from "State i" to "State j"
# Note matrix must be square (i length is j length)

# non-zero and non-one transition states have ranges
# ranges have beta distributions according to paper
zero_to_one = np.random.beta(0.001, 1.000) # Low value is not zero because of documentation
zero_to_zero = (1.000) - (zero_to_one)
one_to_two = np.random.beta(0.001, 0.929)
one_to_three = (1.000) - (one_to_two)
three_to_seven = np.random.beta(0.001, 1.000)
three_to_four = (1.000) - (three_to_seven)
four_to_five = np.random.beta(0.001, 1.000)
four_to_seven = (1.000) - (four_to_five)
five_to_six = np.random.beta(0.001, 0.531)
five_to_eight = np.random.beta(0.001, 0.084)
five_to_seven =(1.000) - (five_to_six) - (five_to_eight)
six_to_eight = np.random.beta(0.001, 0.426)
six_to_seven = (1.000) - (six_to_eight)

trprob_range = np.array([
    [zero_to_zero, zero_to_one, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, one_to_two, one_to_three, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, three_to_four, 0, 0, three_to_seven, 0],
    [0, 0, 0, 0, 0, four_to_five, 0, four_to_seven, 0],
    [0, 0, 0, 0, 0, 0, five_to_six, five_to_seven, five_to_eight],
    [0, 0, 0, 0, 0, 0, 0, six_to_seven, six_to_eight],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
])

### Probabilistic No-Intervention

In [17]:
'''
Monte Carlo Method to repeat simulations
Simply "tallying"
the amount of times individuals appeared in each state
'''

# Number of simulations
simulations = 10**4
n = 20  # Number of steps per simulation

# Initialize count array
state_counts = [0, 0, 0, 0, 0, 0, 0, 0, 0]

# Run simulations
for _ in range(simulations):
    curr_state = 0  # Start from Susceptible
    state_counts[curr_state] += 1  # Count the initial state
    
    steps = n - 1  # Remaining steps
    while steps > 0:
        curr_state = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], p=trprob_range[curr_state])
        state_counts[curr_state] += 1
        steps -= 1

# Print results
for idx, count in enumerate(state_counts):
    print(f"{state[idx]}: {count}")

Susceptible: 200000
Exposed: 0
Infectious_0: 0
Infectious_1: 0
Infectious_2: 0
Infectious_3: 0
Infectious_4: 0
Recovered: 0
Dead: 0


In [18]:
scts = state_counts

# gamma according to paper, but will use normal for now
i1_cost = np.random.uniform(low=92.86, high=154.77)
i2_cost = np.random.uniform(low=9234.96, high=15391.60)
i3_cost = np.random.uniform(low=30175.42, high=50292.88)
i4_cost = np.random.uniform(low=30175.42, high=50292.88)

state_cost_totals = [
    scts[0]*0, 
    scts[1]*0, 
    scts[2]*0, 
    scts[3]*i1_cost, 
    scts[4]*i2_cost, 
    scts[5]*i3_cost, 
    scts[6]*i4_cost, 
    scts[7]*0, 
    scts[8]*0, 
]

state_cost_totals

[0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0]

In [19]:
total_cost = sum(state_cost_totals)
total_cost

0.0

In [20]:
# beta according to paper
sus_util = np.random.beta(0.018, 1.000)
exp_util = np.random.beta(0.018, 1.000)
i0_util = np.random.beta(0.017, 1.000)
i1_util = np.random.beta(0.012, 1.000)
i2_util = np.random.beta(0.010, 0.990)
i3_util = np.random.beta(0.005, 0.495)
i4_util = np.random.beta(0.001, 0.099)
rec_util = np.random.beta(0.018, 1.000)


state_util_totals = [
    scts[0]*sus_util, 
    scts[1]*exp_util, 
    scts[2]*i0_util, 
    scts[3]*i1_util, 
    scts[4]*i2_util, 
    scts[5]*i3_util, 
    scts[6]*i4_util, 
    scts[7]*0, 
    scts[8]*rec_util, 
]

state_util_totals

[6.139186099200873e-22, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0]

In [21]:
total_utility = sum(state_util_totals)
total_utility

6.139186099200873e-22

### Probabilistic Vaccine

In [22]:
'''
Monte Carlo Method for vaccines
'''

# only 4000 people because 60% of total pop is vaccinated
vacc_simulations = 4**3
k = 20

# Initialize with 6000 people already recovered
v_state_counts = [0, 0, 0, 0, 0, 0, 0, 120000, 0]

# Run simulations
for _ in range(vacc_simulations):
    v_curr_state = 0  # Start from Susceptible
    v_state_counts[v_curr_state] += 1  # Count the initial state
    
    v_steps = k - 1  # Remaining steps
    while v_steps > 0:
        v_curr_state = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8], p=trprob_range[v_curr_state])
        v_state_counts[v_curr_state] += 1
        v_steps -= 1

# Print results
for idx, count in enumerate(v_state_counts):
    print(f"{state[idx]}: {count}")

Susceptible: 1280
Exposed: 0
Infectious_0: 0
Infectious_1: 0
Infectious_2: 0
Infectious_3: 0
Infectious_4: 0
Recovered: 120000
Dead: 0


In [23]:
vcts = v_state_counts

vacc_cost_totals = [
    vcts[0]*0, 
    vcts[1]*0, 
    vcts[2]*0, 
    vcts[3]*i1_cost, 
    vcts[4]*i2_cost, 
    vcts[5]*i3_cost, 
    vcts[6]*i4_cost, 
    vcts[7]*0, 
    vcts[8]*0, 
]

vacc_cost_totals

[0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0]

In [24]:
vacc_cost = sum(vacc_cost_totals) + 6000*(np.random.uniform(low=49.40, high=82.34))
vacc_cost

376785.35095213057

In [25]:
vacc_util_totals = [
    vcts[0]*sus_util, 
    vcts[1]*exp_util, 
    vcts[2]*i0_util, 
    vcts[3]*i1_util, 
    vcts[4]*i2_util, 
    vcts[5]*i3_util, 
    vcts[6]*i4_util, 
    vcts[7]*0, 
    vcts[8]*rec_util, 
]

vacc_util_totals

[3.929079103488559e-24, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0]

In [26]:
vacc_utility = sum(vacc_util_totals) + 120000*(np.random.beta(0.018, 1.000))
vacc_utility

1229.9254607915723

### Probabilistic ICERs

In [27]:
det_num = vacc_cost - total_cost
det_deno = vacc_utility - total_utility

det_ICER = det_num/det_deno
det_ICER

306.34811861658176

In [28]:
print("cost delta =", det_num)
print("QALY delta =", det_deno)

cost delta = 376785.35095213057
QALY delta = 1229.9254607915723


## TO DO

This is so far a determistic model, need to do probabilistic model (ranges of variable values = transition probs, costs, utilities)

Also need to incorporate vaccine interventions, change to variables -- might just mean a similar model but different parameters.

Still need to figure out how to change duration per section and then simulate over a year.
Current model would be over 20 days for 10,000 "people"

Need to figure out how to implement gamma distributions